# Data exploration - checking some initial thoughts and ideas, seeing what the data looks like

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def normalise_series(series):
    '''Normalise the values in a series to the bounds of 0 and 1
    
    Parameters
    ----------
    series : pandas series
        series to noramlise
        
    Returns
    -------
    norm_series : pandas series
        normalised version of series
    '''
    norm_series = (series-min(series)) / (max(series)-min(series))
    return norm_series

In [2]:
with open('data/training_set_VU_DM_2014.csv', 'r') as csvfile:
    train = pd.read_csv(csvfile)
# with open('data/test_set_VU_DM_2014.csv', 'rb') as csvfile:
#     test = pd.read_csv(csvfile)

In [ ]:
print(
    train.prop_id.unique().shape, train.shape
)
train_subset = train.sample(frac=0.25)
print(
    train_subset.prop_id.unique().shape, train_subset.shape
)
with open('data/training_set_VU_DM_2014_SUBSET.csv', 'w') as csvfile:
    train_subset.to_csv(csvfile)

(129113,) (4958347, 54)
(104688,) (1239587, 54)


# Feature creation

In [3]:
print(train.columns)

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

## User > search > search output > selection columns

In [ ]:
user_cols = ['srch_id', 'visitor_location_country_id', 'visitor_hist_starrating', 'visitor_hist_adr_usd']
search_cols = ['srch_id', 'date_time', 'site_id', 'srch_destination_id', 'srch_length_of_stay', \
'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', \
'srch_saturday_night_bool', 'srch_query_affinity_score']
search_output_cols = ['srch_id', 'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', \
'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv', \
'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', \
'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv', \
'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', \
'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff', 'orig_destination_distance', \
'random_bool', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', \
'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', \
'price_usd', 'promotion_flag']
selection_cols = ['srch_id', 'click_bool', 'gross_bookings_usd', 'booking_bool', 'position']

In [ ]:
user_train_df = train[user_cols]
search_train_df = train[search_cols]
search_output_train_df = train[search_output_cols]
selection_train_df = train[selection_cols]
user_test_df = test[user_cols]
search_test_df = test[search_cols]
search_output_test_df = test[search_output_cols]

## Preprocessing steps

#### Impute missing values, normalise numerical values, check and correct any potential class imbalances

## 1. Impute missing values - sort out NaN and onehot encode the comp_inv and comp_rate variables

In [3]:
# for the comp1_rate, comp1_inv must one hot encode
list_of_onehots = ['comp1_rate', 'comp1_inv', 'comp2_rate', 'comp2_inv', \
                   'comp3_rate', 'comp3_inv', 'comp4_rate', 'comp4_inv', \
                   'comp5_rate', 'comp5_inv', 'comp6_rate', 'comp6_inv', \
                   'comp7_rate', 'comp7_inv', 'comp8_rate', 'comp8_inv', \
                   'srch_adults_count', 'srch_children_count', \
                   'srch_room_count', 'position']

# first find another value for the NaN to be encoded by, chosen -2 here. 
train[list_of_onehots] = train[list_of_onehots].fillna(-2)

onehots = train[list_of_onehots].copy()

df_with_dummies = pd.get_dummies(onehots,columns=list_of_onehots)
train = train.drop(labels=list_of_onehots,axis=1)
train = pd.concat([train,df_with_dummies], axis=1)

# some null/nan values can be replaced with 0. These are listed below
nan_to_zeros_list = ['visitor_hist_starrating', 'visitor_hist_adr_usd', \
                     'prop_location_score1', 'prop_location_score2', \
                     'promotion_flag', 'srch_query_affinity_score', \
                     'comp1_rate_percent_diff', 'comp2_rate_percent_diff', \
                     'comp3_rate_percent_diff', 'comp4_rate_percent_diff', \
                     'comp5_rate_percent_diff', 'comp6_rate_percent_diff', \
                     'comp7_rate_percent_diff', 'comp8_rate_percent_diff', \
                     'gross_bookings_usd', 'prop_review_score', \
                     'orig_destination_distance'] 

train[nan_to_zeros_list] = train[nan_to_zeros_list].fillna(0)

## 2. Normalising the numerical values to within the bounds of 0 --> 1

In [4]:
norm_list = ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_starrating', \
             'prop_review_score', 'prop_location_score1', 'prop_location_score2', \
             'prop_log_historical_price', 'price_usd', 'srch_length_of_stay', \
             'srch_booking_window', 'srch_query_affinity_score', \
             'orig_destination_distance', 'gross_bookings_usd', \
             'comp1_rate_percent_diff', 'comp2_rate_percent_diff', \
             'comp3_rate_percent_diff', 'comp4_rate_percent_diff', \
             'comp5_rate_percent_diff', 'comp6_rate_percent_diff', \
             'comp7_rate_percent_diff', 'comp8_rate_percent_diff']

for var in norm_list:
    train[var] = normalise_series(train[var])

In [37]:
for var in norm_list:
    print(var, min(train[var]), max(train[var]), np.mean(train[var]))

visitor_hist_starrating 0.0 1.0 0.0342807802681014
visitor_hist_adr_usd 0.0 1.0 0.004585256753408697
prop_starrating 0.0 1.0 0.6361050971220851
prop_review_score 0.0 1.0 0.7951134890989549
prop_location_score1 0.0 1.0 0.41154564449560527
prop_location_score2 0.0 1.0 0.10171327238694666
prop_log_historical_price 0.0 1.0 0.6953161540540906
price_usd 0.0 1.0 1.2886817549381124e-05
srch_length_of_stay 0.0 1.0 0.02473977574726878
srch_booking_window 0.0 1.0 0.07616700219433373
srch_query_affinity_score 0.0 1.0 0.9952667660277239
orig_destination_distance 0.0 1.0 0.07542012982956114
gross_bookings_usd 0.0 1.0 6.768286628694967e-05
comp1_rate_percent_diff 0.0 1.0 0.00015307240767866692
comp2_rate_percent_diff 0.0 1.0 1.2281918218110822e-05
comp3_rate_percent_diff 0.0 1.0 1.2954416713922676e-05
comp4_rate_percent_diff 0.0 1.0 4.627590163954688e-06
comp5_rate_percent_diff 0.0 1.0 8.33474539039434e-06
comp6_rate_percent_diff 0.0 1.0 0.00020654122965168704
comp7_rate_percent_diff 0.0 1.0 5.483659